## Configuração de ambiente

- Primeiro tem que instalar o Postgre na máquina, acesse [postgresql.org](https://www.postgresql.org/download/) para fazer o download na sua máquina.
- Depois de fazer a instalação, você vai ter o usuário padrão "postgres" e a senha que você criou durante a instalação, esse usuário e senha vai ser utilizado para acessar o banco de dados.
- Instale a biblioteca ``pip install psycopg2``
  

#### Acesso ao PostgreSQL via linha de comando
- **Mac**: ```psql -U postgres -W```

- **macOS e Linux**: ```sudo -u postgres psql```

- Vai pedir a senha que você criou na instalação

### Gerenciamento do Banco de Dados

- Esse comando vai criar o banco de dados na sua máquina: ```CREATE DATABASE mds_squad03; ```
  
- Esse comando vai conectar você ao banco de dados criado: ```\c mds_squad03```

- Esse comando vai criar a tabela dentro do banco de dados:
    ```
        CREATE TABLE extracoes (
            id serial PRIMARY KEY,
            municipio TEXT,
            valor DECIMAL(10, 2),
            termo TEXT
        );
    ```

- Digite esse comando pra sair do psql: ``\q``

- Depois é so executar o codigo, que ele vai inserir os dados no banco de dados


### insersão dos dados no banco de dados

In [2]:
import os
import re
import psycopg2

try:
    # Conecte-se ao PostgreSQL
    conn = psycopg2.connect(
        user="postgres", ## aqui é usuário padrão
        password="password", ## senha que você criou  na instalação
        host="localhost", ## como ta na sua máquina é localhost
        database="mds_squad03" # nome da base de dados que foi criado antes
    )
    cur = conn.cursor()

    conn.autocommit = False

    diretorio = r'D:\UnB\MDS\2023-2-Squad03\BOTS\database\txt'

    for arquivo in os.listdir(diretorio):
        if arquivo.endswith('.txt'):
            caminho_arquivo = os.path.join(diretorio, arquivo)

            with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo_txt:
                conteudo = arquivo_txt.read()

            # dividi o txt usando esse separador ---------------------------
            trechos = re.split(r'[-]{27}[:][0-9A-F]+', conteudo)

            for trecho in trechos:
                # aqui é pra varrer se em cada trecho separado tem qualquer valor
                valores = re.findall(r'R\$\s[0-9,.]+', trecho)

                if valores:
                    # esse era pra pegar o codigo do separador mas não funcionou ainda
                    codigo_match = re.search(r'[-]+:([A-F0-9]+)', trecho)
                    codigo = codigo_match.group(1).strip() if codigo_match else ''

                    # aqui é pra pegar o nome do municipio no arquivo
                    municipio = os.path.splitext(arquivo)[0]

                    # Aqui ele puxa os dados que estão no começo de cada trecho
                    termo_match = re.search(r'[A-Z\s]+', trecho)
                    termo = termo_match.group().strip() if termo_match else ''

                    # aqui insere os dados na tabela
                    for valor in valores:
                        valor = valor.replace('R$', '').replace('.', '').replace(',', '.').strip()

                        cur.execute("INSERT INTO extracoes (municipio, valor, termo) VALUES (%s, %s, %s)",
                                    (municipio, valor, termo))
    
except Exception as e:
    # Em caso de exceção, faça o rollback da transação
    if conn:
        conn.rollback()
    print(f"Erro: {e}")
    
finally:
    # Certifique-se de fechar a conexão
    if conn:
        conn.close()


### listar a tabela

In [3]:
import psycopg2

try:
    # Conecte-se ao PostgreSQL
    conn = psycopg2.connect(
        user="postgres", ## aqui é usuário padrão
        password="password", ## senha que você criou  na instalação
        host="localhost", ## como ta na sua máquina é localhost
        database="mds_squad03" # nome da base de dados que foi criado antes
    )
    
    # Crie um cursor
    cur = conn.cursor()
    
    # consulta para recuperar os primeiros registros da tabela
    cur.execute("SELECT * FROM extracoes LIMIT 10;")
    
    # Recupere os resultados
    resultados = cur.fetchall()
    
    # Exiba os resultados
    for resultado in resultados:
        print(resultado)

except Exception as e:
    print(f"Erro: {e}")

finally:
    # Certifique-se de fechar a conexão
    if conn:
        conn.close()


(1292, 'Arapiraca', Decimal('47416.40'), 'PREFEITURA MUNICIPAL DE ARAPIRACA \nAVISO DE HOMOLOGA')
(1293, 'Arapiraca', Decimal('212184.72'), 'PREFEITURA MUNICIPAL DE ARAPIRACA \nTERMO DE RATIFICA')
(1294, 'Arapiraca', Decimal('2694887.56'), 'PREFEITURA MUNICIPAL DE ARAPIRACA \nATA DE JULGAMENTO DE HABILITA')
(1295, 'Arapiraca', Decimal('360.00'), 'PREFEITURA MUNICIPAL DE ARAPIRACA \nAVISO DE HOMOLOGA')
(1296, 'Arapiraca', Decimal('28080.00'), 'PREFEITURA MUNICIPAL DE ARAPIRACA \nAVISO DE HOMOLOGA')
(1297, 'Arapiraca', Decimal('639.50'), 'PREFEITURA MUNICIPAL DE ARAPIRACA \nAVISO DE HOMOLOGA')
(1298, 'Arapiraca', Decimal('19046.00'), 'PREFEITURA MUNICIPAL DE ARAPIRACA \nAVISO DE HOMOLOGA')
(1299, 'Arapiraca', Decimal('10800.00'), 'PREFEITURA MUNICIPAL DE ARAPIRACA \nAVISO DE HOMOLOGA')
(1300, 'Arapiraca', Decimal('25602.00'), 'PREFEITURA MUNICIPAL DE ARAPIRACA \nAVISO DE HOMOLOGA')
(1301, 'Arapiraca', Decimal('180.00'), 'PREFEITURA MUNICIPAL DE ARAPIRACA \nPORTARIA GP N')
